In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re

In [29]:
url = 'https://en.wikipedia.org/wiki/World_Happiness_Report'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>World Happiness Report - Wikipedia</title>
<script>document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"World_Happiness_Report","wgTitle":"World Happiness Report","wgCurRevisionId":903800060,"wgRevisionId":903800060,"wgArticleId":42576223,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Happiness indices","Sustainability metrics and indices"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort

In [30]:
table = soup.find_all('table',{'class':'wikitable sortable'})[0]
table

<table class="wikitable sortable">
<tbody><tr valign="top">
<th style="width: 10px;">Overall rank
</th>
<th style="width: 250px;">Country or region
</th>
<th><abbr title="Happiness score">Score</abbr>
</th>
<th style="width: 10px;"><abbr title="Explained by: GDP">GDP per capita</abbr>
</th>
<th style="width: 10px;"><abbr title="Explained by: Social support">Social support</abbr>
</th>
<th style="width: 10px;"><abbr title="Explained by: Healthy life expectancy">Healthy life expectancy</abbr>
</th>
<th style="width: 10px;"><abbr title="Explained by: Freedom to make life choices">Freedom to make life choices</abbr>
</th>
<th style="width: 10px;"><abbr title="Explained by: Generosity">Generosity</abbr>
</th>
<th style="width: 10px;"><abbr title="Explained by: Perceptions of corruption">Perceptions of corruption</abbr>
</th></tr>
<tr>
<td>1</td>
<td><span class="flagicon"><img alt="" class="thumbborder" data-file-height="1100" data-file-width="1800" decoding="async" height="14" src="//uploa

In [31]:
#finding the rows in the text above
rows = table.find_all('tr')
rows = [row.text.strip().split("\n") for row in rows]
rows

[['Overall rank',
  '',
  'Country or region',
  '',
  'Score',
  '',
  'GDP per capita',
  '',
  'Social support',
  '',
  'Healthy life expectancy',
  '',
  'Freedom to make life choices',
  '',
  'Generosity',
  '',
  'Perceptions of corruption'],
 ['1',
  '\xa0Finland',
  '7.632',
  '1.305',
  '1.592',
  '0.874',
  '0.681',
  '0.192',
  '0.393'],
 ['2',
  '\xa0Norway',
  '7.594',
  '1.456',
  '1.582',
  '0.861',
  '0.686',
  '0.286',
  '0.340'],
 ['3',
  '\xa0Denmark',
  '7.555',
  '1.351',
  '1.590',
  '0.868',
  '0.683',
  '0.284',
  '0.408'],
 ['4',
  '\xa0Iceland',
  '7.495',
  '1.343',
  '1.644',
  '0.914',
  '0.677',
  '0.353',
  '0.138'],
 ['5',
  '\xa0\xa0Switzerland',
  '7.487',
  '1.420',
  '1.549',
  '0.927',
  '0.660',
  '0.256',
  '0.357'],
 ['6',
  '\xa0Netherlands',
  '7.441',
  '1.361',
  '1.488',
  '0.878',
  '0.638',
  '0.333',
  '0.295'],
 ['7',
  '\xa0Canada',
  '7.328',
  '1.330',
  '1.532',
  '0.896',
  '0.653',
  '0.321',
  '0.291'],
 ['8',
  '\xa0New Zealand

In [32]:
#removing the spaces in the columns list (rows[0])
rows[0] = list(filter(lambda x: x != '', rows[0]))
rows[0]

['Overall rank',
 'Country or region',
 'Score',
 'GDP per capita',
 'Social support',
 'Healthy life expectancy',
 'Freedom to make life choices',
 'Generosity',
 'Perceptions of corruption']

In [33]:
colnames = rows[0]
data = rows[1:]
df = pd.DataFrame(data, columns=colnames)
df

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.632,1.305,1.592,0.874,0.681,0.192,0.393
1,2,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340
2,3,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408
3,4,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138
4,5,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357
5,6,Netherlands,7.441,1.361,1.488,0.878,0.638,0.333,0.295
6,7,Canada,7.328,1.330,1.532,0.896,0.653,0.321,0.291
7,8,New Zealand,7.324,1.268,1.601,0.876,0.669,0.365,0.389
8,9,Sweden,7.314,1.355,1.501,0.913,0.659,0.285,0.383
9,10,Australia,7.272,1.340,1.573,0.910,0.647,0.361,0.302
